<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EC%88%98%EC%97%85/Auto_ML_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Seung-Hwan Oh, Seoul, Korea

Modified by Seung-Hwan Oh 2022.12.27

Do not copy without permission

In [1]:
# waring 무시
import warnings
warnings.filterwarnings('ignore')

#Stacking

In [2]:
from sklearn.ensemble import  StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42) # 사이킷런에서 제공해주는 make_mose라는 교육용 데이터
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# base_models 설정
base_models = [
                 ('rf_1', RandomForestClassifier(max_depth=10)),
                 ('rf_2', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy', \
                                                                                   max_depth=1, random_state=1),
                                                                                  learning_rate=0.1, n_estimators=1000))
                ]

# stacking 설정, blender
clf = StackingClassifier(estimators=base_models, \
                         final_estimator=GradientBoostingClassifier(max_depth=4, n_estimators=75, random_state=1))

clf.fit(X_train,y_train)
# final_estimator = blender, 층을 가지는 앙상블 모델을 만들 수 있다 (bagging과 다름)
# base에 catboost를 쓰는게 가장 좋으며 blender에도 catboost를 한 번 더 써줄 수 있다
# bagging에서는 weight를 줘서 sum average를 한다(voting에서는 불가)

StackingClassifier(estimators=[('rf_1', RandomForestClassifier(max_depth=10)),
                               ('rf_2',
                                AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                                                         max_depth=1,
                                                                                         random_state=1),
                                                   learning_rate=0.1,
                                                   n_estimators=1000))],
                   final_estimator=GradientBoostingClassifier(max_depth=4,
                                                              n_estimators=75,
                                                              random_state=1))

In [3]:

from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score,mean_squared_error

test_predictions = clf.predict(X_test)

print("test acc Average: ",accuracy_score(y_test,test_predictions))

print("test Precision Average: ",precision_score(y_test,test_predictions))

print("test Recall Average: ",recall_score(y_test,test_predictions))

print("test F1_score Average: ",f1_score(y_test,test_predictions))

test acc Average:  0.848
test Precision Average:  0.8813559322033898
test Recall Average:  0.8125
test F1_score Average:  0.8455284552845529


# 오토ML-pycaret

In [ ]:
!pip install catboost

In [ ]:
!pip install xgboost

In [ ]:
!pip install pycaret

1) pycaret classification

In [7]:
#pycaret에서 제공하는 'juice' 데이터
from pycaret.datasets import get_data
data = get_data('juice')
data.info()

,Id,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,1,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,2,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,3,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,4,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,5,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070 entries, 0 to 1069
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              1070 non-null   int64  
 1   Purchase        1070 non-null   object 
 2   WeekofPurchase  1070 non-null   int64  
 3   StoreID         1070 non-null   int64  
 4   PriceCH         1070 non-null   float64
 5   PriceMM         1070 non-null   float64
 6   DiscCH          1070 non-null   float64
 7   DiscMM          1070 non-null   float64
 8   SpecialCH       1070 non-null   int64  
 9   SpecialMM       1070 non-null   int64  
 10  LoyalCH         1070 non-null   float64
 11  SalePriceMM     1070 non-null   float64
 12  SalePriceCH     1070 non-null   float64
 13  PriceDiff       1070 non-null   float64
 14  Store7          1070 non-null   object 
 15  PctDiscMM       1070 non-null   float64
 16  PctDiscCH       1070 non-null   float64
 17  ListPriceDiff   1070 non-null   f

In [9]:
# classification
from pycaret.datasets import get_data
from pycaret.classification import * # setup, * = 전체 불러와라
setup_clf = setup(data=data, target='Purchase')
# 각 컬럼에 대한 value가 나타남

,Description,Value
0,Session id,4947
1,Target,Purchase
2,Target type,Binary
3,Target mapping,"CH: 0, MM: 1"
4,Original data shape,"(1070, 19)"
5,Transformed data shape,"(1070, 19)"
6,Transformed train set shape,"(749, 19)"
7,Transformed test set shape,"(321, 19)"
8,Ordinal features,1
9,Numeric features,17


In [10]:
# ML 모델 성능 비교, 상위권 모델 3개 가지고 stacking 하면 된다
# 단, 전처리는 따로 해줘야 한다
top = compare_models()
top

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.8318,0.0000,0.7676,0.7957,0.7794,0.6438,0.6462,0.6250
lr,Logistic Regression,0.8305,0.9063,0.7505,0.8039,0.7730,0.6385,0.6426,1.3220
lda,Linear Discriminant Analysis,0.8238,0.9044,0.7539,0.7880,0.7670,0.6260,0.6301,0.3270
ada,Ada Boost Classifier,0.8211,0.8786,0.7467,0.7852,0.7634,0.6200,0.6226,0.4700
gbc,Gradient Boosting Classifier,0.8211,0.8974,0.7505,0.7846,0.7642,0.6206,0.6240,0.8520
catboost,CatBoost Classifier,0.8157,0.8984,0.7468,0.7749,0.7577,0.6096,0.6127,3.3400
lightgbm,Light Gradient Boosting Machine,0.8117,0.8883,0.7672,0.7586,0.7591,0.6050,0.6092,1.6000
rf,Random Forest Classifier,0.8077,0.8890,0.7333,0.7663,0.7457,0.5919,0.5960,1.5660
xgboost,Extreme Gradient Boosting,0.8064,0.8781,0.7534,0.7535,0.7512,0.5930,0.5955,0.3560
et,Extra Trees Classifier,0.7957,0.8704,0.7266,0.7434,0.7316,0.5674,0.5709,1.5710


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, positive=False, random_state=4947, solver='auto',
                tol=0.0001)

2) pycaret regression

In [11]:
from pycaret.regression import *
from pycaret.datasets import get_data
dataset = get_data('diamond')
exp = setup(dataset, target='Price')
reg_top = compare_models()
reg_top

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


,Description,Value
0,Session id,4312
1,Target,Price
2,Target type,Regression
3,Original data shape,"(6000, 8)"
4,Transformed data shape,"(6000, 29)"
5,Transformed train set shape,"(4200, 29)"
6,Transformed test set shape,"(1800, 29)"
7,Ordinal features,1
8,Numeric features,1
9,Categorical features,6


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,623.2341,1802778.9045,1298.5070,0.9833,0.0660,0.0485,2.1350
xgboost,Extreme Gradient Boosting,691.6285,2081292.5793,1405.6196,0.9803,0.0719,0.0533,1.1250
et,Extra Trees Regressor,761.7089,2588692.7833,1555.2514,0.9761,0.0793,0.0590,2.0720
rf,Random Forest Regressor,755.9153,2751427.7251,1605.3422,0.9743,0.0786,0.0580,1.8740
gbr,Gradient Boosting Regressor,904.0743,3053058.9146,1725.1052,0.9711,0.1013,0.0767,0.6660
lightgbm,Light Gradient Boosting Machine,759.6654,3142252.3960,1710.8568,0.9709,0.0777,0.0566,0.9140
dt,Decision Tree Regressor,975.7541,4400290.0897,2052.0915,0.9579,0.1025,0.0742,0.4940
llar,Lasso Least Angle Regression,2497.7189,14904764.4868,3828.7159,0.8604,0.6475,0.2938,0.2960
ridge,Ridge Regression,2502.8954,14944827.7039,3833.3958,0.8601,0.6421,0.2948,0.3090
br,Bayesian Ridge,2506.1661,15102868.2956,3852.8750,0.8587,0.6354,0.2950,0.5620


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

3) pycaret csv 응용

In [ ]:
# pycaret 버전
import pycaret
print('PyCaret: %s' % pycaret.__version__)

PyCaret: 3.0.4


In [ ]:
from pandas import read_csv
# pycaret classification 모듈
from pycaret.classification import setup
# pycaret classification 모델들을 비교하기 위한 모듈
from pycaret.classification import compare_models
# sonar 데이터셋
# url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/sonar.csv'
# 데이터셋 로드
# df = read_csv(url, header=None)
# 로컬 데이터셋 로드
df = read_csv("data.csv")
# 컬럼 갯수
n_cols = df.shape[1]
# 컬럼명 String으로 반환
df.columns = [str(i) for i in range(n_cols)]
# 데이터셋 설정
grid = setup(data=df, target=df.columns[-1],  verbose=True)
# 모델 학습
best = compare_models()
# best 모델 확인
print(best)

,Description,Value
0,Session id,813
1,Target,60
2,Target type,Binary
3,Target mapping,"M: 0, R: 1"
4,Original data shape,"(208, 61)"
5,Transformed data shape,"(208, 61)"
6,Transformed train set shape,"(145, 61)"
7,Transformed test set shape,"(63, 61)"
8,Numeric features,60
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8557,0.9651,0.7857,0.8986,0.8180,0.7087,0.7286,0.5480
catboost,CatBoost Classifier,0.8343,0.9453,0.7976,0.8819,0.8133,0.6697,0.6978,13.7380
lightgbm,Light Gradient Boosting Machine,0.8276,0.9335,0.7500,0.8856,0.7982,0.6511,0.6748,0.2980
xgboost,Extreme Gradient Boosting,0.8200,0.9215,0.7810,0.8467,0.7996,0.6395,0.6559,0.3890
lr,Logistic Regression,0.8148,0.8478,0.7524,0.8589,0.7763,0.6266,0.6543,0.1520
ada,Ada Boost Classifier,0.8057,0.8721,0.7786,0.8173,0.7867,0.6101,0.6231,0.6110
gbc,Gradient Boosting Classifier,0.8000,0.9145,0.7714,0.8350,0.7798,0.6020,0.6281,0.9020
rf,Random Forest Classifier,0.7933,0.9284,0.7405,0.8556,0.7605,0.5864,0.6241,0.5750
ridge,Ridge Classifier,0.7729,0.0000,0.7238,0.8023,0.7400,0.5437,0.5640,0.1790
svm,SVM - Linear Kernel,0.7633,0.0000,0.7429,0.7953,0.7278,0.5173,0.5614,0.1760


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=-1, oob_score=False,
                     random_state=813, verbose=0, warm_start=False)


# 오토ML-autogluon
- 전처리와 모델 section과 하이퍼파라미터까지 해준다

In [12]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... 

In [13]:
import pandas as pd

df=pd.read_csv("wine.csv", encoding = "utf-8")
display(df.head(10))
display(df.describe())
display(df.info())

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,class
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,1
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5,1
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7,1
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7,1
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,1


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,class
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801,5.818378,0.246114
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712,0.873255,0.430779
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000,3.000000,0.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000,5.000000,0.000000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000,6.000000,0.000000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000,6.000000,0.000000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000,9.000000,1.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  class                 6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB


None

In [16]:
df.select_dtypes(include=object).columns

Index([], dtype='object')

In [17]:
for i in list(df.select_dtypes(include=object).columns):
    print(df[[f'{i}']].value_counts())
    print("=========================")

In [18]:
import pandas as pd
# Tabular Data를 다루기 때문에 아래 라이브러리를 호출
from autogluon.tabular import TabularDataset, TabularPredictor

In [19]:
train_df = pd.read_csv('./wine.csv')

In [20]:
# autogluon 학습을 위한 데이터 형태로 변환
train = TabularDataset(train_df)

In [21]:
#학습
predictor = TabularPredictor(label='class', eval_metric='f1_macro',).fit(train)

No path specified. Models will be saved in: "AutogluonModels/ag-20230821_014556/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230821_014556/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   79.87 GB / 115.66 GB (69.1%)
Train Data Rows:    6497
Train Data Columns: 12
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:       

In [22]:
# 각각의 모델의 훈련 성능을 평가할 수 있음
ld_board = predictor.leaderboard(train, silent=True)

ld_board

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM,0.998962,0.991638,0.119888,0.051360,7.843933,0.119888,0.051360,7.843933,1,True,4
1,XGBoost,0.998755,0.989525,0.056906,0.045202,4.036963,0.056906,0.045202,4.036963,1,True,11
2,LightGBMLarge,0.998755,0.989525,0.187792,0.047422,5.411180,0.187792,0.047422,5.411180,1,True,13
3,RandomForestEntr,0.998755,0.989525,0.366431,0.132939,2.094366,0.366431,0.132939,2.094366,1,True,6
4,RandomForestGini,0.998755,0.989525,0.370110,0.213196,5.946147,0.370110,0.213196,5.946147,1,True,5
5,ExtraTreesEntr,0.998755,0.989525,0.458580,0.177280,2.985401,0.458580,0.177280,2.985401,1,True,9
6,ExtraTreesGini,0.998755,0.989525,0.508983,0.188084,3.034390,0.508983,0.188084,3.034390,1,True,8
7,CatBoost,0.996885,0.989525,0.017005,0.008038,2.147336,0.017005,0.008038,2.147336,1,True,7
8,NeuralNetTorch,0.996672,0.991638,0.072376,0.030926,17.401215,0.072376,0.030926,17.401215,1,True,12
9,NeuralNetFastAI,0.996466,0.993742,0.191098,0.038301,16.636224,0.191098,0.038301,16.636224,1,True,10


# 하이퍼파라미터 튜닝 - 베이지안옵티마이제이션

In [25]:
from sklearn.metrics import r2_score
import xgboost as xgb

# 탐색 대상 함수 (XGBRegressor) 정의해야 함 : 데이터 불러오기. 데이터 쪼개기. 모델 학습, 모델 vaild한 score를 return
def XGB_cv(max_depth,learning_rate, n_estimators):

    # 모델 정의 (float 값이 있기 때문에 int값만 받는 파라미터는 int로 형변환 필요)
    model = xgb.XGBRegressor(max_depth=int(max_depth),
                            learning_rate=learning_rate,
                            n_estimators=int(n_estimators)
                            )
    # 모델 훈련
    model.fit(X_train, y_train)

    # 예측값 출력
    y_pred= model.predict(X_test)
    r2 = r2_score(y_test, y_pred)

    # 오차 최적화로 사용할 metric 반환 (베이지안은 이것을 성능수치로 받아들이게됨)
    return r2

In [29]:
#  bayesian-optimization 라이브러리의 BayesianOptimization 클래스 import
from bayes_opt import BayesianOptimization
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 실험해보고자하는 hyperparameter 집합
# max_depth(int, default: 3): 기본 학습자를 위한 최대 트리 깊이
# learning_rate(float, default: 0.1) : Boosting 학습률
# n_estimators(int, default: 100) : fit하기 위한 Boosted tree의 수

# max_depth, learning_rate, n_estimators 범위 지정
pbounds = {'max_depth': (3, 7),
            'learning_rate': (0.001, 0.2),
            'n_estimators': (5000, 10000)
            }

# Bayesian optimization 객체 생성
# f : 탐색 대상 함수, pbounds : hyperparameter 집합
# verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
bo=BayesianOptimization(f=XGB_cv, pbounds=pbounds, verbose=2, random_state=1 )

# 메소드를 이용해 최대화 과정 수행
# init_points :  초기 Random Search 갯수
# n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
# acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용
# xi : exploration 강도 (기본값은 0.0)
bo.maximize(init_points=2, n_iter=10)

# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냄.
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우,
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인.

# 찾은 파라미터 값 확인
print(bo.max)

ModuleNotFoundError: ignored

# 하이퍼파라미터 튜닝-옵튜나

ref:https://dacon.io/codeshare/2704

In [ ]:
!pip install optuna

In [31]:
import numpy as np
import pandas as pd
import optuna
from lightgbm import LGBMClassifier
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

train_df = pd.read_csv('./wine.csv')
X = train_df.iloc[:,:-1]
y= train_df.iloc[:,-1]

# def objective(Trial) -> float: 와 아래는 동일한 코드
def objective(trial: Trial) -> float:
    params_lgb = {
        "random_state": 42,
        "verbosity": -1,
        "learning_rate": 0.05,
        "n_estimators": 10000,
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 3e-5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 9e-2),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2,shuffle=True)

    # ** 딕셔너리 언패킹
    model = LGBMClassifier(**params_lgb)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        early_stopping_rounds=10,
        verbose=False
    )

    lgb_pred = model.predict_proba(X_valid)
    log_score = log_loss(y_valid, lgb_pred)

    return log_score

sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

[I 2023-08-21 01:55:08,966] A new study created in memory with name: lgbm_parameter_opt
[W 2023-08-21 01:55:08,993] Trial 0 failed with parameters: {'reg_alpha': 1.12424581642324e-05, 'reg_lambda': 0.08556428806974939, 'max_depth': 15, 'num_leaves': 154, 'colsample_bytree': 0.4936111842654619, 'subsample': 0.40919616423534183, 'subsample_freq': 1, 'min_child_samples': 88, 'max_bin': 380} because of the following error: TypeError("LGBMClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-31-a0c8cffa6b7d>", line 36, in objective
    model.fit(
TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'
[W 2023-08-21 01:55:09,007] Trial 0 failed with value None.


TypeError: ignored

In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [32]:
# 언패킹 (리스트)
slist = [1,2,3,4,5]
print(*slist) # 하나씩 떼줌
print(*slist, sep='@')

1 2 3 4 5
1@2@3@4@5


In [40]:
# 언패킹(딕셔너리)
sdict = {'one':1, 'two':2}

def LGBMClassifiers(one, two):
    print(one, two)

LGBMClassifiers(**sdict)

1 2


# 숙제
다음 코드의 모델을 optuna 또는 bayesian optimization을 사용하여 최적의 하이퍼파라미터로 제출하시오.